# Hex words

From: https://github.com/pyhawaii/talks/blob/master/puzzles/boston_pug/hex_words.md

This puzzle comes from the Boston Python User Group.

Paraphrasing from Hex Words:

Certain words can be generated using the six alpha characters used by the hexadecimal numbering system:

a, b, c, d, e, and f.

For our purposes and so that everyone is using the same dictionary use the words file in this repo's boston_pug folder.

Find all English hex() words, i.e. strings of hex digits that appear to be English words. Each hex value can be converted into a decimal value.

For example...

The word bee in hex is 3054 in decimal.
The word face in hex is 64206 in decimal.
The word decade in hex is 14600926 in decimal.
Find the hex word with the highest decimal value.

For those who are interested, I pulled the words list off a Mac from /usr/share/dict/words

Share your solutions with us, either via pull request OR email them to info@pyahawaii.com

In [ ]:
import string
from collections import defaultdict

We'll make a solution that works for any base 11 through 36.

`a=10
b=11
c=12
...`

Strictly speaking we won't need the below, but it seems handy.

In [ ]:
MAX_POWER = 20 # The longest word you expect to consider. 20 should be overkill

In [ ]:
base=16
values = dict(zip(string.ascii_lowercase[:base - 10],range(10, base)))
powers = dict(zip(range(MAX_POWER), (base**i for i in range(MAX_POWER))))
def word_value(word, base=16):
    """
    word a string of lower case letters
    base a number in range(10,37)
    interprets the word as digits in the base and returns the value
    
    >>> word_value('a')
    10
    >>> word_value('aa')
    170
    >>> word_value('bee')
    3054
    >>> word_value('face')
    64206
    >>> word_value('decade')
    14600926
    """
    return sum(powers[i]*values[char] for i,char in enumerate(reversed(word)))
    
    

We'll propose two solutions, one for efficiency and one for aesthetic value.

Either way the strategy is the same: We find the alphabetically highest word of maximal length.

We assume `words.txt` is in the current directory, a text file with one word per line. We also assume the words are in case-insensitive alphabetical order.

## First solution: Designed for aesthetic

We pass through words more than once unnecessarily and consider the full list unnecessarily.

But since the code takes a fraction of a second, these indulgences are fine and indeed correct. Never optimize prematurely.

In [ ]:
digits = {'\n'}.union(set(string.ascii_lowercase[:base - 10]),
                      set(string.ascii_uppercase[:base - 10]))
# lowercase, capital letters and a newline allowed

In [ ]:
with open('words.txt') as f:
    words = [word.strip().lower() for word in f if set(word) <= digits]
# 
word_lengths = defaultdict(list) # {length:list of words}
for word in words:
    word_lengths[len(word)].append(word)
#
max_length = max(word_lengths)
#
theword = max(word_lengths[max_length])
theword, word_value(theword)

## Second solution: Designed for efficiency

We insert more work-saving checks along the way. Every check should save time but add ugliness.

This is an inferior solution.

In [ ]:
digits = set(string.ascii_lowercase[:base - 10])

word_lengths = defaultdict(list) # {length:list of words}
max_length = 0
try:
    next_letter = string.ascii_lowercase[base-10]
except IndexError:
    # Should happen for base 36
    next_letter = 'A' 
with open('words.txt') as f:
    for word in f:
        word = word.strip().lower()
        if word.startswith(next_letter):
            break
        if set(word) <= digits:
            l = len(word)
            if l >= max_length:
                max_length = l
                word_lengths[l].append(word)
theword=word_lengths[max_length][-1]
theword, word_value(theword)

Note that if the text file were not given in alphabetical order, the result would instead be:

`max(word_lengths[max_length])`

For the given word list, neither matters, as there is only one word of length 8!

To consider: Our solution rejected any word containing special characters like spaces of hypens. For this word list, that turned out not to matter. 